In [1]:
from azureml.core import Workspace, Dataset, Datastore, Experiment

In [2]:
ws=Workspace.from_config("./config")

In [3]:
ws=Workspace.from_config("./config")
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

In [ ]:
files_list=["./data/test.csv","./data/test1.csv"]
files_list= az_store.upload_files(files=files_list,
                                   target_path="folder_name/",
                                relative_root="./data/",
                                overwrite=True)

In [ ]:
az_store.upload(src_dir="./data",
                target_path="folder_name/data",
                overwrite=True)

# Creating Experiment

In [4]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_exp1_vikas")

# Running Experiment from Local with script file

# Creating Environment and attach to configuration

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret','azureml-explain-model','interpret-community'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
script_config = ScriptRunConfig(source_directory=".",
                                script="TrainingScript.py",
                                environment=myenv) 
new_run1 = new_experiment.submit(config=script_config)
new_run1.wait_for_completion()

# Creating Compute cluster in azure

In [7]:
from azureml.core.compute import AmlCompute 
cluster_name= "comput1"
if cluster_name not in ws.compute_targets:
    compute_config= AmlCompute.provisioning_configuration(vm_size="",
                                                        max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster= ws.compute_targets[cluster_name]
print(cluster," compute cluster found.. using it....")

AmlCompute(workspace=Workspace.create(name='aml-msft-ml-poc1', subscription_id='a4df42c9-c1d2-44c4-b2c2-05edb4b27aad', resource_group='RG-MSFT-ML-POC-01'), name=comput1, id=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.MachineLearningServices/workspaces/aml-msft-ml-poc1/computes/comput1, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags={})  compute cluster found.. using it....


In [5]:
input_ds=ws.datasets.get("Titanic_selected_columns")

# Running Experiment in azure

In [8]:
from azureml.train.automl import AutoMLConfig
automl_config= AutoMLConfig(task='classification',
                            compute_target=cluster,
                            training_data=input_ds,
                            validation_size=0.3,
                            label_column_name="Survived",
                            primary_metric="norm_macro_recall",
                            iterations=10,
                            max_concurrent_iterations=2,
                            experiment_timeout_hours=0.25,
                            featurization='auto')

In [9]:
from azureml.core.experiment import Experiment
new_exp= Experiment(ws,"azureml_titanic_vikas")
print("submitting the exxperiment")
new_run= new_exp.submit(automl_config)
new_run.wait_for_completion(show_output=True)

submitting the exxperiment
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
azureml_titanic_vikas,AutoML_821a400b-4034-4d54-92e9-bfb3f9400860,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
azureml_titanic_vikas,AutoML_821a400b-4034-4d54-92e9-bfb3f9400860,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization


{'runId': 'AutoML_821a400b-4034-4d54-92e9-bfb3f9400860',
 'target': 'comput1',
 'status': 'Completed',
 'startTimeUtc': '2022-11-03T16:29:10.294294Z',
 'endTimeUtc': '2022-11-03T16:39:20.05976Z',
 'services': {},
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0.3',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'comput1',
  'AMLSettingsJsonString': '{"path":null,"name":"azureml_titanic_vikas","subscription_id":"a4df42c9-c1d2-44c4-b2c2-05edb4b27aad","resource_group":"RG-MSFT-ML-POC-01","workspace_name":"aml-msft-ml-poc1","region":"eastus","compute_target":"comput1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":true,"i

In [ ]:
best_child_run= new_run.get_best_child()

In [ ]:
for run in new_run.get_children():
    print("run id: ",run.id)
    print("accuracy is: ",run.get_metrics['accuracy'])
    print("norm macro recall: ",run.get_metrics['norm_macro_recall'])